<a href="https://colab.research.google.com/github/tyukei/WCNSProject/blob/main/ConnectWCNP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WCN
## Reference
- whisper
https://note.com/k_masaki/n/na6a523648616

- ChatGPT
https://note.com/nero1014/n/n09a2ce7843b8

- Notion
https://yshr10ic.com/2022/01/10/notion-google-colab/


## TODO

```python
def GetOpenaiApi():
  return '<CPTAPI>'

def GetNotionToken():
  return '<NOTIONTOKEN>'   

def GetNotionDB():
  return '<NOTIONDBID>'

```

1. make ```key.py``` and copy paste this code at drive/MyDrive/ColabNotebooks

1. Get ChatGPT API
https://platform.openai.com/account/api-keys

1. Copy Paste Chat GPT API ```'<CPTAPI>'```

1. Add Integration https://www.notion.so/my-integrations

1. Copy Paste Notion Token ```'<NOTIONTOKEN>'```

1. Make DataBase in Notion and get id
  
  ex) `https://www.notion.so/<Username>/<DataBaseID>?v=<ViewID>`

1. Copy Paste Notion DataBase ID ```'<NOTIONDBID>'```

1. Set Other parameters

* WHISPER_MODEL = [base, small, large] \\
base is quick but not correct so much \\
large is good score but takes long time

  

# Setup Environment

## Parameters

In [ ]:
WHISPER_MODEL ="large"
ZOOM_FILE_PATH = "/content/drive/MyDrive/Zoom"
VOICE_LANGUAGE ="ja"
GPT_ROLL="あなたは書記です。音声データは次のものです。" 
GPT_PROMPT_SUMMARY="要約を書いてください。"
GPT_PROMPT_TITLE="内容を一言で表してください"

## Install Whisper

In [ ]:
!pip install git+https://github.com/openai/whisper.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-eqp6ow9r
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-eqp6ow9r
  Resolved https://github.com/openai/whisper.git to commit c09a7ae299c4c34c5839a76380ae407e7d785914
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 62.7 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230314-py3-none-any.whl size=798038 sha256=67066311e33f688d2a0ee9fe6d690e4f5db7180898c90733737f66a499c44fcc
  Stored in directory: /tmp/pip-ephem-wheel-cache-1zie8n3t/wheels/fe/03/29/e7919208d11b4ab32972cb448bb84a9a675d92cd52c9a48341
Successfully built openai-whisper


In [ ]:
import whisper

## Install Chatgpt

In [ ]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 12.3 MB/s eta 0:00:00


In [ ]:
import openai

## Import necessary libraries

In [73]:
import os.path
import warnings
warnings.simplefilter('ignore')
import datetime
import json
from pprint import pprint
import requests
import numpy as np
import pandas as pd
import glob
import textwrap

In [35]:
from google.colab import drive
from google.colab import files
import sys
drive.mount('/content/drive')
sys.path.append('/content/drive/MyDrive/Key/')
import makekey
openai.api_key  = makekey.GetOpenaiApi()
NOTION_TOKEN = makekey.GetNotionToken()
NOTION_DB = makekey.GetNotionDB()
SLACK_WEBHOOK = makekey.GetSlackAPI()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Build Whisper model

In [ ]:
model = whisper.load_model(WHISPER_MODEL)

100%|█████████████████████████████████████| 2.87G/2.87G [00:47<00:00, 65.7MiB/s]


## Build Notion setting

In [ ]:
class Config:
    notion_token = NOTION_TOKEN
    notion_endpoint = 'https://api.notion.com/v1'
    database_id = NOTION_DB
    headers = {
        'Authorization': f'Bearer {notion_token}',
        'Content-Type': 'application/json',
        'Notion-Version': '2021-08-16'
    }

class PropertyType:
    TITLE = 'title'
    RICH_TEXT = 'rich_text'
    NUMBER = 'number'
    DATE = 'date'
    CHECKBOX = 'checkbox'
    SELECT = 'select'
    MULTI_SELECT = 'multi_select'
    RELATION = 'relation'

class Cell:
    def __init__(self, key, properties, values):
        self.key = key
        self.property_type = properties[self.key]
        self.values = values

    def to_dict(self):
        if self.property_type in [PropertyType.TITLE, PropertyType.RICH_TEXT]:
            return self._to_dict_text()
        elif self.property_type == PropertyType.DATE:
            return self._to_dict_pattern2('start')
        elif self.property_type == PropertyType.NUMBER:
            return self._to_dict_pattern1(round(self.values, 5))
        elif self.property_type == PropertyType.CHECKBOX:
            return self._to_dict_pattern1(self.values)
        elif self.property_type == PropertyType.SELECT:
            return self._to_dict_pattern2('name')
        elif self.property_type == PropertyType.MULTI_SELECT:
            values = self.get_multi_select_values()
            return self._to_dict_pattern1(values)
        elif self.property_type == PropertyType.RELATION:
            return self._to_dict_relation()

    def _to_dict_text(self):
        return {
            self.key: {
                self.property_type: [
                    {
                        'text': {
                            'content': self.values
                        }
                    }
                ]
            }
        }

    def _to_dict_relation(self):
        return {
            self.key: {
                self.property_type: [
                    {
                        'id': self.get_page_id()
                    }
                ]
            }
        }

    def _to_dict_pattern1(self, values):
        return {
            self.key: {
                self.property_type: values
            }
        }

    def _to_dict_pattern2(self, name):
        return {
            self.key: {
                self.property_type: {
                    name: self.values
                }
            }
        }

    def get_multi_select_values(self):
        values = []
        if isinstance(self.values, str):
            values.append({'name': self.values})
        else:
            for val in self.values:
                values.append({'name': val})
        return values
    
    def get_page_id(self):
        body = {
            'filter': {
                'property': 'Exp',
                'text': {
                    'contains': self.values
                }
            }
        }
        url = f'{Config.notion_endpoint}/databases/{Config.database_id}/query'
        response = requests.request('POST', url=url, headers=Config.headers, data=json.dumps(body))

        return response.json()['results'][0]['id']

def to_notion(cells):
    props = {}
    
    for cell in cells:
        props.update(cell.to_dict())
    
    body = {
        'parent': {
            'database_id': Config.database_id
        },
        'properties': props,
    }

    url = f'{Config.notion_endpoint}/pages'
    return requests.request('POST', url=url, headers=Config.headers, data=json.dumps(body))

# Run Whisper, Chatgpt and Notion

In [ ]:
def RunWhisper():
  m4a_files = glob.glob(ZOOM_FILE_PATH + "/*/*.m4a")
  if m4a_files[0] is None:
    return 'null'
  elif not os.path.isfile(m4a_files[0]):  
    print('not find file')
    return 'null'
  else:
    print("Get " + m4a_files[0])
    print('Creating Text by Whisper')
    result = model.transcribe(m4a_files[0], language=VOICE_LANGUAGE)
    print(result["text"])
    return result["text"]


In [ ]:
def GetGPT(role,content):
  result = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0301",
    messages=[
      {"role": role, "content": content},
    ]
  )
  text_result= result['choices'][0]['message']['content']

In [ ]:
def RunGPT(whisper_result):
  print('Creating Summary and Title by ChatGPT')
  result_summary = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0301",
    messages=[
      {"role": "system", "content": GPT_ROLL+ whisper_result},
      {"role": "user", "content":GPT_PROMPT_SUMMARY}
    ]
  )
  text_summary = result_summary['choices'][0]['message']['content']
  print("Summary:"+ text_summary)
  result_title = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0301",
    messages=[
      {"role": "assistant", "content": result_summary['choices'][0]['message']['content']},
      {"role": "user", "content":GPT_PROMPT_TITLE + whisper_result}
    ]
  )
  text_title = result_title['choices'][0]['message']['content']
  print("Title:" + text_title)
  return [text_summary,text_title]

In [77]:
def RunSendNotion(whisper_result,gpt_result):
  print('Creating Notion Page')
  response = requests.request('GET', url=f'{Config.notion_endpoint}/databases/{Config.database_id}', headers=Config.headers) 
  properties = {}
  for k, v in response.json()['properties'].items():
      properties[k] = v['type']
  jst = datetime.datetime.now() + datetime.timedelta(hours=9)
  cells = [
      Cell('Name', properties, gpt_result[1]),
      Cell('Date', properties, jst.strftime('%Y%m%d-%H:%M')),
      Cell('Summary', properties, gpt_result[0]),
      Cell('AllContext', properties, whisper_result)
  ]
  result = to_notion(cells)
  # pprint(result)
  return result

In [78]:
def SendSlack(message):
# 任意のメッセージを通知する関数
    # 作成したWebhook URL
    url =  SLACK_WEBHOOK
    data = json.dumps({'text': message})
    headers = {'content-type': 'application/json'}
    requests.post(url, data=data, headers=headers)

def RunSendSlack(notion_result):
    SendSlack(notion_result.json()['properties']['Date']['multi_select'][0]['name'])
    SendSlack(notion_result.json()['properties']['Name']['title'][0]['plain_text'])
    SendSlack(notion_result.json()['properties']['Summary']['rich_text'][0]['plain_text'])
    print('RSended to slack')

In [80]:
whisper_result = RunWhisper()
if whisper_result == 'null':
  exit()
  
gpt_result = RunGPT(whisper_result)
notion_result = RunSendNotion(whisper_result,gpt_result)
if notion_result.status_code == requests.codes.ok:
  RunSendSlack(notion_result)

Get /content/drive/MyDrive/Zoom/2023-04-16 21.53.36 Keita Nakata's Zoom Meeting/audio00.m4a
Creating Text by Whisper
今日は朝おにぎりを食べて、昼にスパゲティを食べました。夜はカレーを食べる予定です。
Creating Summary and Title by ChatGPT
Summary:人物は朝におにぎりを食べ、昼にスパゲティを食べ、夜はカレーを食べる予定である。
Title:一日三食を計画している。
Creating Notion Page
RSended to slack
